In [ ]:
import os
import sys
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2 as cv

# custom lib
from inspect import getsourcefile
import os.path as path, sys
sys.path.insert(0, "..")
from alprLib.Yolo import Yolo
from alprLib.TextOCR import Reader
from alprLib.Perspective import platePerspectiveUnwarpingWithWhite
from alprLib.Perspective import platePerspectiveUnwarpingWithSuperPixel
from alprLib.Perspective import platePerspectiveUnwarping
sys.path.pop(0)


In [ ]:
def clear_output(wait=False):
    from IPython.core.interactiveshell import InteractiveShell
    if InteractiveShell.initialized():
        InteractiveShell.instance().display_pub.clear_output(wait)
    else:
        print('\033[2K\r', end='')
        sys.stdout.flush()
        print('\033[2K\r', end='')
        sys.stderr.flush()

# Video

#### Process video file

In [ ]:
video_path = "video1.mp4"

if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('output/images'):
    os.makedirs('output/images')

count = -1
video_capture = cv.VideoCapture(video_path)

yolo = Yolo()
# reader = Reader()

while(True):
    # Read frame
    ret, frame = video_capture.read()
    if(frame is None): break
    count +=1

    if count % 1==0: # Skip some frames
        # Clear result output
        clear_output(wait=True)

        # Process frame
        bboxes = yolo.find_bboxes(frame)
        rects = []
        corrections = []
        for bbox in bboxes:
            rect = patches.Rectangle(
                (bbox[0], bbox[1]),
                bbox[2], bbox[3],
                linewidth=1, edgecolor='r', facecolor='none')
            rects.append(rect)

            plate = [
                bbox[0], bbox[1],
                bbox[0]+bbox[2], bbox[1]+bbox[3]
            ]

            # corrected = platePerspectiveUnwarpingWithWhite(
            # 	frame,
            #     plate[0], plate[1], plate[2], plate[3],
            # )
            corrected = platePerspectiveUnwarpingWithSuperPixel(
              frame, plate[0], plate[1], plate[2], plate[3],
              plate_padding = 15,
              main_color = np.array([200,200,200]),
              color_dist = 150
            )
            # corrected = platePerspectiveUnwarping(
            #     frame, plate[0], plate[1], plate[2], plate[3],
            #     plate_padding=15
            # )
            corrections.append(corrected)

        # Plot frame
        fig = plt.figure(figsize=(18, 18*frame.shape[0]/frame.shape[1]))

        ax0 = fig.add_subplot(111, aspect='equal')
        ax0.axis('off')

        ax0.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
        for i, rect in enumerate(rects):
            ax0.add_patch(rect)
        for i, rect in enumerate(rects):
            ax1 = fig.add_subplot(1,5,i+1)
            ax1.axis('off')
            ax1.imshow(corrections[i])

        # Save the frame
        plt.savefig(os.path.join('output', 'images', str(count).zfill(6) + '.png'))
        plt.show()



#### Save output video

In [ ]:

image_folder = os.path.join('output', 'images')
video_name = os.path.join('output','output-video.avi')

images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
frame = cv.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv.VideoWriter(video_name, 0, 30, (width,height))

for image in images:
    video.write(cv.imread(os.path.join(image_folder, image)))

video.release()
